<a href="https://colab.research.google.com/github/To-Infinity-Reemas/Piezoelectric/blob/main/Copy_of_map80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pandas numpy scikit-learn folium openpyxl

In [ ]:
import pandas as pd
import numpy as np
import folium
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler

In [ ]:
#ملف اكسل للبيانات الملعب
file_path = "/content/generated_data_24hrar.xlsx"
df = pd.read_excel(file_path)

In [ ]:
#تنظيف البيانات
#حذف القيم الناقصه
df = df.dropna(subset=["ACC_X", "ACC_Y", "Steps"])


In [ ]:
#احصائيات الخطوات
Q1 = df["Steps"].quantile(0.25)
Q2 = df["Steps"].median()
Q3 = df["Steps"].quantile(0.75)
IQR = Q3 - Q1

In [ ]:
#داله لتحديد اللون حسب الخطوات
def get_color(steps):
    if steps < Q1:
        return "blue"      # قليل جدًا
    elif Q1 <= steps < Q2:
        return "cyan"      # قليل
    elif Q2 <= steps < Q3:
        return "green"     # متوسط
    elif Q3 <= steps < (Q3 + 1.5 * IQR):
        return "yellow"    # كثير
    else:
        return "red"       # مستحيل (قيمة شاذة)

In [ ]:
#تجهيز البيانات لمودل DBSCAN
X = df[["ACC_X", "ACC_Y"]].values
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
#تنفيذ مودل DBSCAN
db = DBSCAN(eps=0.3, min_samples=5).fit(X_scaled)
df["Cluster"] = db.labels_

In [ ]:
#معرفة عدد العناقيد
n_clusters = len(set(db.labels_)) - (1 if -1 in db.labels_ else 0)
print(f" clusters {n_clusters}")
print(f" noise (Noise): {list(db.labels_).count(-1)}")

 clusters 5
 noise (Noise): 19


In [ ]:
#تعيين الوان للعناقيد
cluster_colors = ["purple", "orange", "brown", "pink", "gray", "darkgreen", "black", "lightblue", "darkred", "beige"]
def cluster_color(c):
    if c == -1:
        return "black"  # ضجيج
    else:
        return cluster_colors[c % len(cluster_colors)]

In [ ]:
#رسم الخريطه
m = folium.Map(location=[df["ACC_X"].mean(), df["ACC_Y"].mean()], zoom_start=15)

#إضافة المستطيل الأخضر حول جميع النقاط
min_x, max_x = df["ACC_X"].min(), df["ACC_X"].max()
min_y, max_y = df["ACC_Y"].min(), df["ACC_Y"].max()

folium.Rectangle(
    bounds=[(min_y, min_x), (max_y, max_x)],
    color="green",
    fill=True,
    fill_color="green",
    fill_opacity=0.2
).add_to(m)

In [ ]:
#رسم الدوائر
for i, row in df.iterrows():
    step_color = get_color(row["Steps"])
    cluster_col = cluster_color(row["Cluster"])

    folium.CircleMarker(
        location=[row["ACC_X"], row["ACC_Y"]],
        radius=6,
        color=cluster_col,
        fill=True,
        fill_color=step_color,
        fill_opacity=0.8,
        popup=folium.Popup(f"""
            <b> الوقت:</b> {row['TIME']}<br>
            <b> خطوات:</b> {row['Steps']}<br>
            <b> الموقع:</b> X: {row['ACC_X']}, Y: {row['ACC_Y']}<br>
            <b> Z:</b> {row['ACC_Z']}, <b>YAW:</b> {row['YAW']}°<br>
            <b> cluste :</b> {row['Cluster']}
        """, max_width=300)
    ).add_to(m)

max_x = df["ACC_X"].max()  # أعلى إحداثي X
max_y = df["ACC_Y"].max()  # أعلى إحداثي Y
offset_x = 0.0005  # يتحكم بعرض الملعب
offset_y = 0.0007  # يتحكم بطول الملعب

bounds = [
    [max_x - offset_y, max_y - offset_x],  # الزاوية السفلية اليسرى
    [max_x + offset_y, max_y + offset_x]   # الزاوية العلوية اليمنى
]



In [ ]:
#توضيح لتدرج الالوان
legend_html = f"""
<div style="
    position: fixed;
    bottom: 30px; left: 30px; width: 280px; height: 210px;
    background-color: white; border:2px solid grey; z-index:9999; font-size:14px;
    padding: 10px;">
<b>شرح التدرج اللوني للخطوات</b><br>
<i style='background:blue; display:inline-block; width:10px; height:10px; margin-right:5px;'></i> خطوة قليلة جداً: أقل من {Q1:.0f}<br>
<i style='background:cyan; display:inline-block; width:10px; height:10px; margin-right:5px;'></i> خطوة قليلة: بين {Q1:.0f} و {Q2:.0f}<br>
<i style='background:green; display:inline-block; width:10px; height:10px; margin-right:5px;'></i> خطوة متوسطة: بين {Q2:.0f} و {Q3:.0f}<br>
<i style='background:yellow; display:inline-block; width:10px; height:10px; margin-right:5px;'></i> خطوة كثيرة: بين {Q3:.0f} و {Q3 + 1.5 * IQR:.0f}<br>
<i style='background:red; color:white; display:inline-block; width:10px; height:10px; margin-right:5px;'></i> خطوة كثيرة جداً: أكثر من {Q3 + 1.5 * IQR:.0f}<br>
<hr style="margin:5px 0">
<b>شرح ألوان الحدود</b><br>
<i style='color:black; border:1px solid black; padding:1px; border-radius:50%; display:inline-block;'>&#9679;</i> noise (Noise)<br>
<i style='color:purple; border:1px solid purple; padding:1px; border-radius:50%; display:inline-block;'>&#9679;</i> cluster 0.0<br>
</div>
"""
m.get_root().html.add_child(folium.Element(legend_html))

In [ ]:
#حفظ الخريطه
m.save("clustered_map12.html")
print(" تم حفظ الخريطة باسم clustered_map.html مع تلوين حسب DBSCAN والخطوات")

 تم حفظ الخريطة باسم clustered_map.html مع تلوين حسب DBSCAN والخطوات
